## NLP Medical Text Disease Confirmation
In this template we are going to be teting mutiple aspects of NLP based founadtion models for the ability to detect when a pateint has been confirmed to have a disease or condition - 

#### The Models We will be using are 

* Roberta - For Setement Analysis 

* Sci spacy and Stanza with Negex for Negation / Confirmation tracking

* And Bert Q and A to see if we can directly get ansers to medical questions 

Lets get started -

## Roberta NLP Sentiment Analyisis 
Our target will be when a cancer pateint transitions to become metastatic if we can identify a differnce in sentiment scores and use it for a form of clsification around the targets

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.0 MB/s eta 0:00:00


In [ ]:
# import what is needed from libs
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

load in the model 

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

print('model loaded')

model loaded


In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

download full model 

In [ ]:
# download and save the model 
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# -tune or retrain the model here and then save with command bellow 

#save the model - put in name of folder in instead to save to target location
model.save_pretrained(MODEL)

In [ ]:
# example of model use 
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
        
    return " ".join(new_text)

In [ ]:
text = "the CT scan came back negative the disease is not spreading"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.5319
2) negative 0.3688
3) positive 0.0993


## Build it All Togther
build it all out into one full application style method for use -

In [ ]:
#!pip install transformers
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
        
    return " ".join(new_text)


def get_sent(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores

  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  res =[]

  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      res.append(f"{i+1}) {l} {np.round(float(s), 4)}")

  return res

In [ ]:
sent = get_sent('the patient shows no sgin of mets their condition is improving')
print(sent)

['1) positive 0.5934', '2) neutral 0.3804', '3) negative 0.0262']


## BERT NLP
Now Lets check out the peformance of another model for sentiment and
see if it is better or worse at disease progesstion tarcking 


In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

result = classifier("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: NEGATIVE, with score: 0.9991
label: POSITIVE, with score: 0.9999


## Build it Out
as with the previous one lets build this out into a functional method for use



In [ ]:
from transformers import pipeline

def get_sent2(text):
  classifier = pipeline("sentiment-analysis")
  result = classifier(text)[0]
  res = f"label: {result['label']}, with score: {round(result['score'], 4)}"
  print(res)

In [ ]:
res = get_sent2('the patient is now found to have metastasis to the bone')
res

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: NEGATIVE, with score: 0.9829


In [ ]:
res = get_sent2('the patient is not found to have metastasis to the bone')
res

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: NEGATIVE, with score: 0.8297


In [ ]:
res = get_sent2('the patient dose not have mets')
res

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: POSITIVE, with score: 0.7665


## Negex + Stanza 
next we will use a high powered medical model plus a powerful language model fro negation tracking to label target medical data and give us info on if it was negated 

In [ ]:
!pip install spacy
!pip install negspacy
!pip install spacy_stanza

In [ ]:
import spacy # to build a nlp pipeline
import stanza # for named entity recognition
# this package wraps Stanza around Spacy, so that we can use Stanza in a spaCy pipeline.
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset # to customize negation terms
import pandas as pd


In [ ]:
# download and initialize a mimic pipeline with an i2b2 NER model
# stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = spacy_stanza.load_pipeline('en', package='mimic', processors={'ner': 'i2b2'})

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


## Add customized terms to the default list of terms

In [ ]:
ts = termset("en_clinical")
# customize the term list by adding more negation terms
ts.add_patterns({
            'preceding_negations': ['abstain from','other than','except for','except','with the exception of',
                                    'excluding','lack of','contraindication','contraindicated','interfere with',
                                   'prohibit','prohibits'],
            'following_negations':['negative','is allowed','impossible','exclusionary']
        })

## Let negex know what entities we are extracting

In [ ]:
nlp.add_pipe("negex", config={"ent_types":["PROBLEM","TEST",'TREATMENT']})


In [ ]:
doc = nlp('Patient had a CT scan, after review we belive they are postive for mets')

for e in doc.ents:
  print(e.label_)
  print(e.text, e._.negex)

TEST
a CT scan False
PROBLEM
mets False


In [ ]:
doc = nlp('Patient had a CT scan, after review we belive they are negative for mets')

for e in doc.ents:
  print(e.label_)
  print(e.text, e._.negex)

TEST
a CT scan True
PROBLEM
mets True


### A major win for our use case 
A first the above result may look like it did a good job understanding mets negation but failes but think again the scan result is in fact and with our current set up a negex result of true means that the target identified is negatted the model is working perfectly and this is just what we need to help us identify metasttaic disease across patients with high level NLP.

### Futher Items Needed 
to finialize this utilization for our specifc use case we will need to 

* impliment it on the last targets of a ptient jouney to get the most relivent results 

* indentify how to tune train or augment the model with our own probllem targets so we can expand not only the forms of negation we need to identify but also the mets based targets we need to check confirmation on.

## Bert Q and A
Last we will test information based extraction with some of the most powerful bert models we have access to in the BERT Q and A model this will give us the most power out of all the models we are looking for and allow us to ask questions to the code directly but the central question we need to answer with the model is can it be useful to our target identifaction needs at a speed that will work at scale.

In [2]:
#transformers
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    answer =''
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "NaN"
    
    #print("\nPredicted answer:\n{}".format(answer.capitalize()))
    return answer

In [4]:
text = 'Ada came to the hospital today with pain in her left shoulder needs a full examiniation'
question = 'patients problem?'

ans = question_answer(question, text)
ans

'pain in her left shoulder'

Now lets target the text toward metastisis and see if we cant identify the presence of the disease 

In [5]:
text = 'current CT scans are positive it is now highly likely that the patient has metastatic cancer'
question = 'patients problem?'

ans = question_answer(question, text)
ans

'metastatic cancer'

In [6]:
text = 'all current CT scans are negative it is clear the patient is non metastatic at this time'
question = 'patients problem?'

ans = question_answer(question, text)
ans

'non metastatic'

## Another Use Case win
this looks great and we can clearly see targets we can use being identified however this approach has more compute cost assosiated with it and we would need to still add in some hard coded logic to transform it into a true patienet clasification at the end. 

The best use case seems to be negex paired with sci -stanza as this gives us full clasifications as well as handels negation tracking for us adtional logic will still be needed but it will run fatser and not need as much as our other potential options. 